In [ ]:
# Settings
filename    = 'D:/Documenten/PhD/Data/run6_hdf/run-6_measurement-012.hdf5'
file_format = 'hfd5'
loglevel    = 'DEBUG'
peak_types  = [b's1', b's2', b'unknown', b'noise', b'lone_pulse']

In [ ]:
# Import modules
# We get a log file
import logging
log = logging.getLogger('XAMS_analysis')
log.setLevel(loglevel)
# Notebookloader is needed to import ipython notebooks as python modules.
# Notebookloader prompts a warning, but it seems to work fine
# TODO Perhaps we need to reconsider what functions we actually need and put them in a regular python script?
import logging
import NotebookLoader
import Function_definitions as fn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', size=16)
import h5py
import units
from recarray_tools import append_fields

# Optional progress bar
try:
    from tqdm import tqdm
except ImportError:
    log.debug("You don't have tqdm, I can't give you a nice progress bar...")
    def dummy(*args,**kwargs):
        return args[0]
    tqdm = dummy

In [ ]:
##
# Load the data
##
log.debug("Now loading %s (file format=%s)..." % (filename, file_format))

# Slurp peaks and events into memory
# WARNING: For a large dataset, use pax to remove big low-level fields first
# (area_per_channel, does_channel_contribute, does_channel_have_noise)
try:
    # Load the file using pax 3's IO code
    # This supports any format pax can currently produce
    from pax.formats import flat_data_formats
    ioformat = flat_data_formats[file_format]()
    ioformat.open(filename, 'r')
    events = ioformat.read_data('Event')
    peaks = ioformat.read_data('Peak')
    ioformat.close()
   
except ImportError:
    log.debug("You don't have pax 3 installed, falling back to HDF5-specific code...")
    import h5py
    f = h5py.File(filename)
    events = f.get('Event')[:]
    peaks = f.get('Peak')[:]
    f.close()

log.info("Loaded %s, containing %d peaks (%0.2f MB RAM) and %d events (%0.2f MB RAM)" % (
    filename, len(peaks), peaks.nbytes/10**6, len(events), events.nbytes/10**6))
if len(events) == 0:
    raise ValueError("You don't have any events in this dataset!")

In [ ]:
# WORK IN PROGRESS (EH)
# Loop over peaks and redefine the classification
# TODO get this to work
# # At the moment, we just use the default from the processer
# s1_cut    = (peaks['type'] == b's1')
# s2_cut    = (peaks['type'] == b's2')
# other_cut = np.logical_not(s1_cut | s2_cut)

# #peaks = append_fields(peaks,'true_type',np.zeros([len(peaks)],dtype=np.str))

# n_peaks = len(peaks)
# n_s1_before = len(peaks[peaks['type'] == b's1'])

# for pk in peaks:
#     if pk['type'] == b's1':
#         pk['true_type'] = 's1'
    
# peaks[s2_cut]['type']
# log.debug('Before reclassification: %.2f%% of the peaks is an S1' % (n_s1_before/n_peaks*100))


In [ ]:
# Append fields to the event and peak list. Only the ones that can be directly determined from the events lists

# full range: the total time of the peak.
peaks = append_fields(peaks, 'full_width', (peaks['right'] - peaks['left'] + 1)*fn.dt)
peaks = append_fields(peaks,'mid',peaks['left']*fn.dt + 0.5* peaks['full_width'])

# Recalculate area fraction top because of wrong ordering of PMTs
peaks['area_fraction_top'] = peaks['area_per_channel'][:,3] / peaks['area']

In [ ]:
# Group peaks by event
peaks_per_event = fn.group_by(peaks, 'Event')
#assert len(peaks_per_event) == len(events) # Raise error if false

# Add total number of peaks
events = append_fields(events,
                       'n_peaks',
                       np.array([len(x) for x in peaks_per_event]))

# Add number of individual peak types
for pt in peak_types:
    events = append_fields(events,
                           'n_'+pt.decode(),
                           np.array([len(x[x['type'] == pt]) for x in peaks_per_event]))
# Add number of BIG peaks (mainly S2 is important). Use to cut pileup / double scatters    
    events = append_fields(events,
                           'n_big_'+pt.decode(),
                           np.array([len(x[(x['type'] == pt) & (x['area'] >= 100)]) for x in peaks_per_event]))

In [ ]:
##
# Select events with >=1 S1 and >=1 S2
# Necessary step for next cell
##
n_before = len(events)
cut = (events['n_s1'] >= 1) & (events['n_s2'] >= 1)
events = events[cut]
peaks_per_event = [x for i, x in enumerate(peaks_per_event) if cut[i]]

log.debug("%0.2f%% of events have >=1 S1 and S2, keeping only those." % (100*len(events)/n_before))
if((100*len(events)/n_before)<50):
    log.warning("Only %0.2f%% of events have >=1 S1 and S2, keeping only those." % (100*len(events)/n_before))
    

In [ ]:
##
# Add s1_... and s2_... fields to event
# This way you can access all properties of the main S1 and S2 directly from the event
# TODO add peak area per channel
##
ignore_fields = ['Event', 'Peak', 'type',
                 # recfunctions have trouble with subarrays...
                 'does_channel_contribute', 'does_channel_have_noise', 'area_per_channel']
for pt in (b's1', b's2'):
    main_peaks = []
    for pks in tqdm(peaks_per_event, desc='Selecting %ss' % pt.decode()):
        peaks_of_this_type =  pks[pks['type'] == pt]
        main_peaks.append(
            peaks_of_this_type[
                np.argmax(
                   peaks_of_this_type['area']
                )
            ])
    main_peaks = np.array(main_peaks)
    assert len(main_peaks) == len(peaks_per_event)

    # Add s1_... and s2_... fields
    for fn in main_peaks.dtype.names:
        if fn in ignore_fields:
            continue
        events = append_fields(events,
                               "%s_%s" % (pt.decode(), fn),
                               main_peaks[fn])
        
if(len(peaks['type']==b'NaI') > 0):
    log.debug('Found external NaI, will add those properties!')
    main_peaks = []
    for pks in tqdm(peaks_per_event, desc='Selecting NaI pulses'):
        peaks_of_this_type =  pks[pks['detector'] == b'NaI']
        main_peaks.append(
            peaks_of_this_type[
                np.argmax(
                   peaks_of_this_type['area']
                )
            ])
    main_peaks = np.array(main_peaks)
    assert len(main_peaks) == len(peaks_per_event)

    # Add NaI_... fields
    for fn in main_peaks.dtype.names:
        if fn in ignore_fields:
            continue
        events = append_fields(events,
                               "NaI_%s" % fn ,
                               main_peaks[fn])
else:
   log.debug('No NaI found, so just TPC channels added.')


In [ ]:
# Add drift time
events = append_fields(events, 'drift_time', events['s2_hit_time_mean'] - events['s1_hit_time_mean'])
# Drift time is here from hit time mean to hit time mean. Its the mean of the hit maximum weighted by area.
# This DOES introduce a constant, but it's probably a very stable quantity.

In [ ]:
# That was the main analysis code! Now we get a couple of example examples of plots
# Maybe we should move this to some other file?
#
#
# Examples: - a histogram with legend and a double scale
#           - a scatterplot with coloured scale
#           - a density plot

In [ ]:
# Define a cut. Examples given: cut away negative drift time, take dirft time slice or cut events with more than 2 big S2s
# cut = (events['drift_time'] > 0 * units.us)
# cut = (events['drift_time'] > 30 * units.us) & (events['drift_time'] < 40 * units.us)
cut = events['drift_time']>-1000000000
log.info('Cut acceptance: %.2f%%' % (len(events[cut])/len(events)*100))

In [ ]:
# Plot number of S2s for each waveform, and also the number of BIG S2s.
plt.hist(events['n_big_s2'],bins=9,range=(0,8),histtype='step',label='Big S2s')
plt.hist(events['n_s2'],bins=9,range=(0,8),histtype='step',label='Any S2')
plt.legend()
plt.show()

In [ ]:
# Drift time v. S2 width
plt.figure(figsize=(10,7))
plt.scatter(events[cut]['drift_time'] / units.us, events[cut]['s2_full_width'] / units.us,
            marker='.', edgecolor='None', alpha=1, s=10, 
            c=np.log10(events[cut]['s2_area']))
# Alpha: opacity (0: transparent)
# s = size of marker (A.U.)
plt.xlim(0,80)
plt.ylim(0,30)
plt.xlabel('Drift time (us)')
plt.ylabel('S2 width (right-left, us)')
plt.colorbar(label='log10 S2 area (pe)')
plt.show()

In [ ]:
plt.hist(events['drift_time'],bins=100,range=(0,70000))
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hexbin(events['drift_time']/units.us,events['s2_area'],extent=[0, 70, 100, 15000],gridsize = 100)
# See also: plt.pcolor (square bins)    pcolormesh (in comb with np.hist2d)
plt.xlabel('Drift time (us)')
plt.ylabel('S2 area (right-left, us)')
plt.colorbar()
plt.show()

In [ ]:
from nice_plots import ColoredScatterPlot, DensityPlot
# Did not get y log scale to work yet
# TODO do this
DensityPlot(events, dataset_name='events', x='drift_time', y='s2_area',x_range=(0,70000),y_range=(1000,200000)).plot()

In [ ]:
plt.figure(figsize=(10,7))
plt.hexbin(events['drift_time']/units.us,events['s1_area_fraction_top'],extent=[0, 70, 0, 0.4],gridsize = 200)
plt.xlabel('Drift time (us)')
plt.ylabel('S1 area fraction top')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hexbin(events['drift_time']/units.us,events['s1_area'],extent=[0, 70, 0, 1000],gridsize = 200)
plt.xlabel('Drift time (us)')
plt.ylabel('S1 area')
plt.colorbar()
plt.show()

In [ ]:
drift_cut = (events['drift_time'] > 20 * units.us) & (events['drift_time'] < 30 * units.us)

plt.figure(figsize=(10,7))
plt.hist(events[drift_cut]['s1_area'],bins=100,range=(0,1000))
plt.xlabel('S1 area (p.e)')
plt.ylabel('Events/bin')
plt.show()

In [ ]:
supercut = (events['drift_time'] > 20 * units.us) & (events['drift_time'] < 40 * units.us) & (events['s1_area'] > 0)
plt.figure(figsize=(10,7))
plt.hexbin(events[supercut]['s1_area'],events[supercut]['s2_area'],extent = [0,400,0,20000],gridsize = 100,bins='log')
plt.xlabel('S1 area (pe)')
plt.ylabel('S2 area (pe)')
plt.show()


In [ ]:
# Determine electron lifetime

In [ ]:
timestep = 2 * units.us
starttime = 10 * units.us
endtime  = 50 * units.us
# Array of center times of bin
times =np.array([ time + 0.5*timestep for time in np.arange(starttime,endtime,timestep)])
# list of cuts for all these bins
timebins = [ (events['drift_time'] > time) & (events['drift_time'] < time + timestep) 
                                            for time in np.arange(starttime,endtime,timestep) ]
# Array of average for this cut
avgs = np.array([np.average(events[timebins[i]]['s2_area']) for i in range(int((endtime-starttime)/timestep)) ])

In [ ]:
# Should show a decreasing trend
plt.scatter(times / units.us,avgs)
plt.show()

In [ ]:
# We fit a power of e to this...
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * np.exp(-(1/b) * x)
popt, pcov = curve_fit(func, times,avgs,p0 = [12000.,2 * units.us])
perr = np.sqrt(np.diag(pcov))
log.info('Determined electron lifetime: %.2f +/- %.2f us.' % (popt[1]/units.us,perr[1]/units.us))

In [ ]:
# And plot it! 
# TODO add a quantity: scaled S2
fitx = np.linspace(0,15 * units.us,61)
fity = func(fitx,popt[0],popt[1])

plt.figure(figsize = (10,7))
plt.scatter(times/units.us,avgs)
plt.plot(fitx/units.us,fity)
plt.xlim(0,20)
plt.ylim(0,13000)
plt.xlabel('Drift time (us)')
plt.ylabel('S2 area (p.e.)')
plt.show()

In [ ]:
from nice_plots import ColoredScatterPlot, DensityPlot

In [ ]:
ColoredScatterPlot(events[cut], dataset_name='events', y='s2_full_width', z='s2_hit_time_std',x='drift_time',
                   y_range=(0,40*10**3), x_range=(0*10**3,80*10**3), z_range=(0,1000), 
                   ).plot()

In [ ]:
DensityPlot(events[cut], dataset_name='events', x='s2_left', y='s2_right', count_logscale=True).plot()

In [ ]:
ColoredScatterPlot(events[cut], dataset_name='events', x='drift_time', y='s2_full_width', z='n_s2',
                   y_range=(0,80000),z_range=(0,5)).plot()

In [ ]:
ColoredScatterPlot(events,dataset_name = 'events',x='s1_area', y='s2_area',z='drift_time',
                  x_logscale=False,y_logscale=False,
                  x_range=(0,1000),y_range=(0,40000), grainsize=3, grainalpha=0.5).plot()